In [1]:
import frost2df # for frost.help
from frost2df import frost2df, obs2df, lightning2df, codetable
import datetime
import numpy as np
import folium
from folium.features import DivIcon
from pyproj import CRS
from pyproj import Transformer
import requests
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display


client_id = 'b8b1793b-27ff-4f4d-a081-fcbcc5065b53'
client_secret = '7f24c0ca-ca82-4ed6-afcd-23e657c2e78c'

In [2]:
df_stasjoner = frost2df('sources', {'county': '*møre*'})
df_stasjoner['validFrom'] = pd.to_datetime(df_stasjoner['validFrom'], format="%Y-%m-%d")
mask = (df_stasjoner['validFrom'] > '1700-01-01') & (df_stasjoner['validFrom'] <= '1980-01-01')
df_utvalg= df_stasjoner.loc[mask]
df_utvalg.reset_index(inplace=True)
df_utvalg

,index,@type,id,name,shortName,country,countryCode,masl,validFrom,county,...,municipality,municipalityId,stationHolders,externalIds,wigosId,geometry.@type,geometry.coordinates,geometry.nearest,wmoId,icaoCodes
0,3,SensorSystem,SN59800,SVINØY FYR,Svinøy fyr,Norge,NO,38,1955-06-01 00:00:00+00:00,MØRE OG ROMSDAL,...,HERØY,1515,[MET.NO],"[0-20000-0-01205, 10.249.1.56]",0-20000-0-01205,Point,"[5.268, 62.3293]",False,1205.0,NaN
1,8,SensorSystem,SN62270,MOLDE LUFTHAVN,Molde LH,Norge,NO,3,1972-03-01 00:00:00+00:00,MØRE OG ROMSDAL,...,MOLDE,1506,[AVINOR],"[0-20000-0-01217, 10.249.0.60, ENML]",0-20000-0-01217,Point,"[7.2617, 62.7443]",False,1217.0,[ENML]
2,10,SensorSystem,SN59680,ØRSTA-VOLDA LUFTHAMN,Ørsta-Volda LH,Norge,NO,74,1971-06-01 00:00:00+00:00,MØRE OG ROMSDAL,...,ØRSTA,1520,[AVINOR],"[0-20000-0-01209, 10.249.0.153, 10.249.0.58, E...",0-20000-0-01209,Point,"[6.078056927162791, 62.180413205637954]",False,1209.0,[ENOV]
3,16,SensorSystem,SN64330,KRISTIANSUND LUFTHAVN,Kristiansund LH,Norge,NO,62,1970-06-01 00:00:00+00:00,MØRE OG ROMSDAL,...,KRISTIANSUND,1505,[AVINOR],"[0-20000-0-01223, 10.249.0.12, ENKB]",0-20000-0-01223,Point,"[7.8158, 63.1093]",False,1223.0,[ENKB]
4,19,SensorSystem,SN62700,HUSTADVATN,Hostadvatn,Norge,NO,80,1923-01-01 00:00:00+00:00,MØRE OG ROMSDAL,...,HUSTADVIKA,1579,[MET.NO],NaN,0-578-0-62700,Point,"[7.2436, 62.9087]",False,NaN,NaN
5,22,SensorSystem,SN62290,MOLDE - NØISOMHED,Molde − Nøisomhed,Norge,NO,5,1974-04-04 00:00:00+00:00,MØRE OG ROMSDAL,...,MOLDE,1506,[MOLDE KOMMUNE],[10.249.1.68],0-578-0-62290,Point,"[7.2275, 62.7413]",False,NaN,NaN
6,24,SensorSystem,SN64300,KRISTIANSUND - KARIHOLA,Karihola,Norge,NO,5,1973-09-27 00:00:00+00:00,MØRE OG ROMSDAL,...,KRISTIANSUND,1505,[NVE],[110.1],0-578-0-64300,Point,"[7.7027, 63.1257]",False,NaN,NaN
7,25,SensorSystem,SN62900,EIDE PÅ NORDMØRE,Eide,Norge,NO,49,1923-01-01 00:00:00+00:00,MØRE OG ROMSDAL,...,HUSTADVIKA,1579,[MET.NO],NaN,0-578-0-62900,Point,"[7.3905, 62.8915]",False,NaN,NaN
8,29,SensorSystem,SN65370,SMØLA - MOLDSTAD,Smøla,Norge,NO,30,1963-10-01 00:00:00+00:00,MØRE OG ROMSDAL,...,SMØLA,1573,[MET.NO],NaN,0-578-0-65370,Point,"[8.063, 63.4197]",False,NaN,NaN
9,31,SensorSystem,SN60400,NORDDAL,Norddal,Norge,NO,28,1895-07-01 00:00:00+00:00,MØRE OG ROMSDAL,...,FJORD,1578,[MET.NO],NaN,0-578-0-60400,Point,"[7.2392, 62.2477]",False,NaN,NaN


In [3]:
print(df_utvalg['validFrom'][1].date())

1972-03-01


In [4]:

m = folium.Map(location=[df_utvalg['geometry.coordinates'][0][1], df_utvalg['geometry.coordinates'][0][0]], zoom_start=8) #Bruker transformerte koordinater (ikkej funne ut korleis bruke UTM med folium)

for i in range(len(df_utvalg)):
    folium.Marker(
        location=[df_utvalg['geometry.coordinates'][i][1], df_utvalg['geometry.coordinates'][i][0]],
        popup=df_utvalg['name'][i] + ' | ' + df_utvalg['id'][i] + ' | ' + str(df_utvalg['validFrom'][i]),
        icon=folium.Icon(color="green")
    ).add_to(m)
#Legger til Norgeskart WMS

folium.raster_layers.WmsTileLayer(
    url='https://opencache.statkart.no/gatekeeper/gk/gk.open_gmaps?layers=topo4&zoom={z}&x={x}&y={y}',
    name='Norgeskart',
    max_bounds=True,
    fmt='image/png',
    layers='topo4',
    attr=u'<a href="http://www.kartverket.no/">Kartverket</a>',
    transparent=True,
    overlay=True,
    control=True,
    
).add_to(m)

m

In [5]:
idliste = ['SN62290', 'SN62270 ', 'SN60800', 'SN60990']

In [6]:
df = df_stasjoner[df_stasjoner['id'].isin(idliste)]
df.reset_index(inplace=True)
df

,index,@type,id,name,shortName,country,countryCode,masl,validFrom,county,...,municipality,municipalityId,stationHolders,externalIds,wigosId,geometry.@type,geometry.coordinates,geometry.nearest,wmoId,icaoCodes
0,22,SensorSystem,SN62290,MOLDE - NØISOMHED,Molde − Nøisomhed,Norge,NO,5,1974-04-04 00:00:00+00:00,MØRE OG ROMSDAL,...,MOLDE,1506,[MOLDE KOMMUNE],[10.249.1.68],0-578-0-62290,Point,"[7.2275, 62.7413]",False,NaN,NaN
1,42,SensorSystem,SN60800,ØRSKOG,Ørskog,Norge,NO,5,1895-07-01 00:00:00+00:00,MØRE OG ROMSDAL,...,ÅLESUND,1507,[MET.NO],[10.249.1.23],0-578-0-60800,Point,"[6.8167, 62.4775]",False,NaN,NaN
2,83,SensorSystem,SN60990,VIGRA,Vigra,Norge,NO,22,1958-07-01 00:00:00+00:00,MØRE OG ROMSDAL,...,GISKE,1532,[AVINOR],"[0-20000-0-01210, 10.249.0.154, 10.249.0.59, E...",0-20000-0-01210,Point,"[6.115, 62.5617]",False,1210.0,[ENAL]


In [7]:
m = folium.Map(location=[df['geometry.coordinates'][0][1], df['geometry.coordinates'][0][0]], zoom_start=9) #Bruker transformerte koordinater (ikkej funne ut korleis bruke UTM med folium)
for i in range(len(df)):
    folium.Marker(
        location=[df['geometry.coordinates'][i][1], df['geometry.coordinates'][i][0]],
        popup=df['name'][i] + ' | ' + df['id'][i] + ' | ' + str(df['validFrom'][i]),
        icon=folium.Icon(color="green")
    ).add_to(m)
#Legger til Norgeskart WMS
folium.raster_layers.WmsTileLayer(
    url='https://opencache.statkart.no/gatekeeper/gk/gk.open_gmaps?layers=topo4&zoom={z}&x={x}&y={y}',
    name='Norgeskart',
    fmt='image/png',
    layers='topo4',
    attr=u'<a href="http://www.kartverket.no/">Kartverket</a>',
    transparent=True,
    overlay=True,
    control=True,
    
).add_to(m)
m

In [8]:
for index, row in df.iterrows():
    print(row['validFrom'].date())
    parameters = {
        'sources': row['id'],
        'elements': 'mean(air_temperature P1D)',
        'referencetime': str(row['validFrom'].date()) + '/2024-04-03',
        'timeoffsets': 'PT6H'
    }
    display(obs2df(parameters=parameters, verbose=True))
    

1974-04-04
https://frost.met.no/observations/v0.jsonld?sources=SN62290&elements=mean%28air_temperature+P1D%29&referencetime=1974-04-04%2F2024-04-03&timeoffsets=PT6H


,sourceId,elementId,referenceTime,value,unit,qualityCode,timeOffset,timeResolution,level.value
0,SN62290:0,mean(air_temperature P1D),2016-05-14 00:00:00+00:00,4.1,degC,2,PT6H,P1D,2
1,SN62290:0,mean(air_temperature P1D),2016-05-15 00:00:00+00:00,5.4,degC,2,PT6H,P1D,2
2,SN62290:0,mean(air_temperature P1D),2016-05-16 00:00:00+00:00,5.6,degC,2,PT6H,P1D,2
3,SN62290:0,mean(air_temperature P1D),2016-05-17 00:00:00+00:00,7.7,degC,2,PT6H,P1D,2
4,SN62290:0,mean(air_temperature P1D),2016-05-18 00:00:00+00:00,5.9,degC,2,PT6H,P1D,2
...,...,...,...,...,...,...,...,...,...
1650,SN62290:0,mean(air_temperature P1D),2022-03-31 00:00:00+00:00,-0.3,degC,2,PT6H,P1D,2
1651,SN62290:0,mean(air_temperature P1D),2022-04-01 00:00:00+00:00,1.2,degC,2,PT6H,P1D,2
1652,SN62290:0,mean(air_temperature P1D),2022-04-02 00:00:00+00:00,1.8,degC,2,PT6H,P1D,2
1653,SN62290:0,mean(air_temperature P1D),2022-04-03 00:00:00+00:00,2.2,degC,2,PT6H,P1D,2


1895-07-01
https://frost.met.no/observations/v0.jsonld?sources=SN60800&elements=mean%28air_temperature+P1D%29&referencetime=1895-07-01%2F2024-04-03&timeoffsets=PT6H


,sourceId,elementId,referenceTime,value,unit,qualityCode,timeOffset,timeResolution,level.value
0,SN60800:0,mean(air_temperature P1D),2014-11-19 00:00:00+00:00,3.1,degC,2,PT6H,P1D,2
1,SN60800:0,mean(air_temperature P1D),2014-11-20 00:00:00+00:00,3.5,degC,2,PT6H,P1D,2
2,SN60800:0,mean(air_temperature P1D),2014-11-21 00:00:00+00:00,3.0,degC,2,PT6H,P1D,2
3,SN60800:0,mean(air_temperature P1D),2014-11-22 00:00:00+00:00,0.1,degC,2,PT6H,P1D,2
4,SN60800:0,mean(air_temperature P1D),2014-11-23 00:00:00+00:00,1.8,degC,2,PT6H,P1D,2
...,...,...,...,...,...,...,...,...,...
2687,SN60800:0,mean(air_temperature P1D),2022-03-31 00:00:00+00:00,0.1,degC,2,PT6H,P1D,2
2688,SN60800:0,mean(air_temperature P1D),2022-04-01 00:00:00+00:00,0.5,degC,2,PT6H,P1D,2
2689,SN60800:0,mean(air_temperature P1D),2022-04-02 00:00:00+00:00,1.1,degC,2,PT6H,P1D,2
2690,SN60800:0,mean(air_temperature P1D),2022-04-03 00:00:00+00:00,1.2,degC,2,PT6H,P1D,2


1958-07-01
https://frost.met.no/observations/v0.jsonld?sources=SN60990&elements=mean%28air_temperature+P1D%29&referencetime=1958-07-01%2F2024-04-03&timeoffsets=PT6H


,sourceId,elementId,referenceTime,value,unit,qualityCode,timeOffset,timeResolution,level.value
0,SN60990:0,mean(air_temperature P1D),1958-07-02 00:00:00+00:00,19.9,degC,2,PT6H,P1D,2
1,SN60990:0,mean(air_temperature P1D),1958-07-03 00:00:00+00:00,17.5,degC,2,PT6H,P1D,2
2,SN60990:0,mean(air_temperature P1D),1958-07-04 00:00:00+00:00,14.1,degC,2,PT6H,P1D,2
3,SN60990:0,mean(air_temperature P1D),1958-07-05 00:00:00+00:00,11.9,degC,2,PT6H,P1D,2
4,SN60990:0,mean(air_temperature P1D),1958-07-06 00:00:00+00:00,11.2,degC,2,PT6H,P1D,2
...,...,...,...,...,...,...,...,...,...
23231,SN60990:0,mean(air_temperature P1D),2022-03-31 00:00:00+00:00,-0.2,degC,2,PT6H,P1D,2
23232,SN60990:0,mean(air_temperature P1D),2022-04-01 00:00:00+00:00,1.3,degC,2,PT6H,P1D,2
23233,SN60990:0,mean(air_temperature P1D),2022-04-02 00:00:00+00:00,1.9,degC,2,PT6H,P1D,2
23234,SN60990:0,mean(air_temperature P1D),2022-04-03 00:00:00+00:00,2.0,degC,2,PT6H,P1D,2


In [9]:
for index, row in df.iterrows():
    print(row['validFrom'].date())
    parameters = {
        'sources': row['id'],
        'elements': 'mean(surface_snow_thickness P1M)',
        'referencetime': str(row['validFrom'].date()) + '/2024-04-03',
        'timeoffsets': 'P1M'
    }
    display(obs2df(parameters=parameters, verbose=True))
    

1974-04-04
https://frost.met.no/observations/v0.jsonld?sources=SN62290&elements=mean%28surface_snow_thickness+P1M%29&referencetime=1974-04-04%2F2024-04-03&timeoffsets=P1M
Error! Returned status code 412
MESSAGE :  412
REASON :  No time series found for this combination of parameters, check /observations/availableTimeSeries for more information.


""


1895-07-01
https://frost.met.no/observations/v0.jsonld?sources=SN60800&elements=mean%28surface_snow_thickness+P1M%29&referencetime=1895-07-01%2F2024-04-03&timeoffsets=P1M
Error! Returned status code 412
MESSAGE :  412
REASON :  No time series found for this combination of parameters, check /observations/availableTimeSeries for more information.


""


1958-07-01
https://frost.met.no/observations/v0.jsonld?sources=SN60990&elements=mean%28surface_snow_thickness+P1M%29&referencetime=1958-07-01%2F2024-04-03&timeoffsets=P1M
Error! Returned status code 412
MESSAGE :  412
REASON :  No time series found for this combination of parameters, check /observations/availableTimeSeries for more information.


""


In [10]:
available = frost2df('observations/availableTimeSeries', {'sources': 'SN60800'}).sort_values('validFrom')
available

,sourceId,validFrom,validTo,timeOffset,timeResolution,timeSeriesId,elementId,unit,performanceCategory,exposureCategory,status,uri,level.levelType,level.unit,level.value,codeTable
153,SN60800:0,1895-07-01T00:00:00.000Z,NaN,PT6H,P1M,0,sum(precipitation_amount_anomaly P1M 1991_2020),percent,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
75,SN60800:0,1895-07-01T00:00:00.000Z,NaN,PT0H,P1M,0,over_time(sum(time_of_maximum_precipitation_am...,Date,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
47,SN60800:0,1895-07-01T00:00:00.000Z,NaN,PT6H,P1M,0,max(sum(precipitation_amount P1D) P1M),mm,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
99,SN60800:0,1895-07-01T00:00:00.000Z,NaN,PT6H,P1M,0,sum(precipitation_amount P1M),mm,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
160,SN60800:0,1895-07-01T00:00:00.000Z,NaN,PT6H,P1M,0,sum(precipitation_amount_anomaly P1M 1961_1990),percent,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,SN60800:0,2020-12-07T07:51:03.000Z,NaN,PT0H,P1M,0,min(relative_humidity P1M),percent,C,1,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
144,SN60800:0,2020-12-07T07:51:03.000Z,NaN,PT0H,P1D,0,min(relative_humidity P1D),percent,C,1,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
139,SN60800:0,2020-12-07T07:51:03.000Z,NaN,PT0H,P1D,0,max(relative_humidity P1D),percent,C,1,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
10,SN60800:0,2020-12-07T07:51:03.000Z,NaN,PT0H,P1D,0,mean(relative_humidity P1D),percent,C,1,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN


In [11]:
available.loc[available['validFrom'] == '1895-07-01T00:00:00.000Z']

,sourceId,validFrom,validTo,timeOffset,timeResolution,timeSeriesId,elementId,unit,performanceCategory,exposureCategory,status,uri,level.levelType,level.unit,level.value,codeTable
153,SN60800:0,1895-07-01T00:00:00.000Z,NaN,PT6H,P1M,0,sum(precipitation_amount_anomaly P1M 1991_2020),percent,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
75,SN60800:0,1895-07-01T00:00:00.000Z,NaN,PT0H,P1M,0,over_time(sum(time_of_maximum_precipitation_am...,Date,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
47,SN60800:0,1895-07-01T00:00:00.000Z,NaN,PT6H,P1M,0,max(sum(precipitation_amount P1D) P1M),mm,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
99,SN60800:0,1895-07-01T00:00:00.000Z,NaN,PT6H,P1M,0,sum(precipitation_amount P1M),mm,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
160,SN60800:0,1895-07-01T00:00:00.000Z,NaN,PT6H,P1M,0,sum(precipitation_amount_anomaly P1M 1961_1990),percent,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
63,SN60800:0,1895-07-01T00:00:00.000Z,NaN,PT6H,P1M,0,max(surface_snow_thickness P1M),cm,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN


In [12]:
available.loc[available['timeResolution'] == 'P1D']

,sourceId,validFrom,validTo,timeOffset,timeResolution,timeSeriesId,elementId,unit,performanceCategory,exposureCategory,status,uri,level.levelType,level.unit,level.value,codeTable
56,SN60800:0,1957-01-01T00:00:00.000Z,NaN,PT6H,P1D,0,sum(precipitation_amount P1D),mm,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
106,SN60800:0,1957-01-01T00:00:00.000Z,NaN,PT6H,P1D,0,surface_snow_thickness,cm,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,NaN
4,SN60800:0,1957-01-01T00:00:00.000Z,2013-01-31T00:00:00.000Z,PT6H,P1D,0,snow_coverage_type,code,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,snow_cover_code
13,SN60800:0,1957-01-05T00:00:00.000Z,2012-12-31T00:00:00.000Z,PT6H,P1D,0,over_time(precipitation_type P1D),code,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,weather_class1_significance
109,SN60800:0,1957-01-05T00:00:00.000Z,2012-12-31T00:00:00.000Z,PT6H,P1D,0,over_time(precipitation_class1 P1D),code,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,NaN,NaN,NaN,weather_class1_significance
95,SN60800:0,2014-11-17T00:00:00.000Z,2020-06-15T00:00:00.000Z,PT0H,P1D,0,mean(air_temperature P1D),degC,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
116,SN60800:0,2014-11-17T00:00:00.000Z,2020-06-15T00:00:00.000Z,PT0H,P1D,0,integral_of_deficit(mean(air_temperature P1D) ...,degree-day,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
120,SN60800:0,2014-11-17T00:00:00.000Z,2020-06-15T00:00:00.000Z,PT0H,P1D,0,integral_of_excess(mean(air_temperature P1D) P...,degree-day,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
76,SN60800:0,2014-11-17T00:00:00.000Z,2020-06-15T00:00:00.000Z,PT0H,P1D,0,integral_of_excess(mean(air_temperature P1D) P...,degree-day,C,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN
6,SN60800:0,2014-11-17T00:00:00.000Z,2017-06-06T08:16:37.000Z,PT0H,P1D,0,mean(relative_humidity P1D),percent,A,2,Authoritative,https://frost.met.noobservations/v0.jsonld?sou...,height_above_ground,m,2.0,NaN


In [13]:
for index, row in df.iterrows():
    print(row['validFrom'].date())
    parameters = {
        'sources': row['id'],
        'elements': 'surface_snow_thickness',
        'referencetime': str(row['validFrom'].date()) + '/2024-04-03',
    }
    display(obs2df(parameters=parameters, verbose=True))

1974-04-04
https://frost.met.no/observations/v0.jsonld?sources=SN62290&elements=surface_snow_thickness&referencetime=1974-04-04%2F2024-04-03


,sourceId,elementId,referenceTime,value,unit,qualityCode,timeOffset,timeResolution
0,SN62290:0,surface_snow_thickness,1979-10-01 00:00:00+00:00,0,cm,2,PT6H,P1D
1,SN62290:0,surface_snow_thickness,1979-10-02 00:00:00+00:00,0,cm,2,PT6H,P1D
2,SN62290:0,surface_snow_thickness,1979-10-03 00:00:00+00:00,0,cm,2,PT6H,P1D
3,SN62290:0,surface_snow_thickness,1979-10-04 00:00:00+00:00,0,cm,2,PT6H,P1D
4,SN62290:0,surface_snow_thickness,1979-10-05 00:00:00+00:00,0,cm,2,PT6H,P1D
...,...,...,...,...,...,...,...,...
2643,SN62290:0,surface_snow_thickness,1986-12-26 00:00:00+00:00,0,cm,2,PT6H,P1D
2644,SN62290:0,surface_snow_thickness,1986-12-27 00:00:00+00:00,0,cm,2,PT6H,P1D
2645,SN62290:0,surface_snow_thickness,1986-12-28 00:00:00+00:00,0,cm,2,PT6H,P1D
2646,SN62290:0,surface_snow_thickness,1986-12-29 00:00:00+00:00,0,cm,2,PT6H,P1D


1895-07-01
https://frost.met.no/observations/v0.jsonld?sources=SN60800&elements=surface_snow_thickness&referencetime=1895-07-01%2F2024-04-03
Error! Returned status code 403
MESSAGE :  Forbidden
REASON :  The estimated number of observations (1128672) exceeds the maximum number allowed (100000)


""


1958-07-01
https://frost.met.no/observations/v0.jsonld?sources=SN60990&elements=surface_snow_thickness&referencetime=1958-07-01%2F2024-04-03


,sourceId,elementId,referenceTime,value,unit,qualityCode,timeOffset,timeResolution
0,SN60990:0,surface_snow_thickness,2005-01-01 00:00:00+00:00,0,cm,2,PT6H,P1D
1,SN60990:0,surface_snow_thickness,2005-01-02 00:00:00+00:00,0,cm,2,PT6H,P1D
2,SN60990:0,surface_snow_thickness,2005-01-03 00:00:00+00:00,0,cm,2,PT6H,P1D
3,SN60990:0,surface_snow_thickness,2005-01-04 00:00:00+00:00,0,cm,2,PT6H,P1D
4,SN60990:0,surface_snow_thickness,2005-01-05 00:00:00+00:00,0,cm,2,PT6H,P1D
...,...,...,...,...,...,...,...,...
5478,SN60990:0,surface_snow_thickness,2020-08-26 00:00:00+00:00,0,cm,0,PT6H,P1D
5479,SN60990:0,surface_snow_thickness,2020-08-27 00:00:00+00:00,0,cm,0,PT6H,P1D
5480,SN60990:0,surface_snow_thickness,2020-08-28 00:00:00+00:00,0,cm,0,PT6H,P1D
5481,SN60990:0,surface_snow_thickness,2020-08-29 00:00:00+00:00,0,cm,0,PT6H,P1D
